In [1]:
from pynq.lib.video import *
import cv2
import numpy as np
import time
from PIL import Image
from IPython.display import display

In [2]:
from pynq import Overlay
overlay = Overlay("./bitstream/sobel_hdmi.bit")

In [3]:
Mode = VideoMode(640,480,8)
hdmi_out = overlay.video.hdmi_out
#hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.configure(Mode, PIXEL_GRAY)
hdmi_out.start()

frame_in_w = 640
frame_in_h = 480

In [4]:
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))

capture device is open: True


In [5]:
num_frames = 100
readError = 0

In [6]:
import pynq.lib.dma

dma_send = overlay.axi_dma_from_ps_to_pl
dma_recv = overlay.axi_dma_from_pl_to_ps

In [7]:
from pynq import Xlnk
import numpy as np
import time
'''
xlnk = Xlnk()
input_buffer = xlnk.cma_array(shape=(frame_in_h,frame_in_w), dtype=np.uint8, cacheable = 1)
output_buffer = xlnk.cma_array(shape=(frame_in_h,frame_in_w), dtype=np.uint8, cacheable = 1)
#globalimage=np.zeros((frame_in_h,frame_in_w,3), dtype=np.uint8)

'''

xlnk = Xlnk()
input_buffer = xlnk.cma_array(shape=(480,640), dtype=np.uint8, cacheable = 1)
output_buffer = xlnk.cma_array(shape=(478,638), dtype=np.uint8, cacheable = 1)
output_buffer_2 = xlnk.cma_array(shape=(480, 640), dtype=np.uint8, cacheable = 1)



/usr/lib/python3/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: pynq.Xlnk is deprecated and will be removed in 2.7 - use pynq.allocate instead
  if sys.path[0] == '':


In [8]:
while True:
    ret, frame_vga = videoIn.read()
    #frame_vga = cv2.resize(frame_vga, (512, 512), interpolation=cv2.INTER_AREA)

    frame_gray = cv2.cvtColor(frame_vga, cv2.COLOR_BGR2GRAY)
    #result = Image.fromarray(frame_gray)
    #display(result)

    input_buffer[0:480*640] = frame_gray
    if (ret):
        outframe = hdmi_out.newframe()
        dma_send.sendchannel.transfer(input_buffer)
        dma_recv.recvchannel.transfer(output_buffer)
        dma_send.sendchannel.wait()
        dma_recv.recvchannel.wait()
        output_buffer_2[1:479, 1:639] = np.where(output_buffer > 0, output_buffer, output_buffer)
        #buf_image_2 = Image.fromarray(out_buffer_2)
        #outframe[:] = cv2.resize(output_buffer, (640, 480), interpolation=cv2.INTER_AREA)
        hdmi_out.writeframe(output_buffer_2)

KeyboardInterrupt: 

In [9]:
del input_buffer,output_buffer, output_buffer_2

In [10]:
hdmi_out.stop()
hdmi_out.close()